In [ ]:
import pandas as pd
from pmdarima.arima import auto_arima
import csv
import os
import numpy as np


def fast_with_auto_arima(sequence):
    stepwise_model = auto_arima(sequence, start_p=0, d=1, start_q=0, max_p=5, max_q=5, max_order=30,
                                # seasonal=False, m=1,
                                # test='adf',
                                trace=False,
                                error_action='ignore',  # don't want to know if an order does not work
                                suppress_warnings=True,  # don't want convergence warnings
                                stepwise=True
                                # ,njob=-1
                                )
    # print('aic:%s' % stepwise_model.aic())

    stepwise_model.fit(sequence)
    future_forecast = stepwise_model.predict(n_periods=12)  # change here if want preted more than one step

    # print(stepwise_model.summary().tables[1])
    # print('future_forecast:%s'%future_forecast)
    return future_forecast


# 处理横向数据
def rowToSequence(filePath):
    try:

        with open(filePath, encoding='UTF-8') as f:

            reader = csv.reader(f, delimiter=',')
            result = {}
            i = 0

            for row in reader:

                if i > 0:
                    rowKey = row[0]
                    detail = []
                    for value in row[1:]: detail.append(int(value))
                    result[rowKey] = detail

                i += 1

        return result

    except Exception as e:

        return {'status': False, 'message': "ERROR: 读取内容失败:%s" % e}


def statistics(source_file, result_file):
    dataList = rowToSequence(source_file)
    # dataList = res['dataList']
    result = []
    allDiffSum = 0
    count = 0
    tu_denominator = 0
    tu_molecule = 0

    for gpno, sequence in dataList.items():
        dataSequence = sequence[:-12]
        actualValue = sequence[-12:]
        yhat = fast_with_auto_arima(dataSequence)
        forecastValue =yhat
        for i in range(len(forecastValue)):
                if forecastValue[i]<0:
                    forecastValue[i]=0
        forecastValue=np.around(forecastValue,2)
        print('预测：{}'.format(forecastValue))
        allDiffSum += abs(actualValue - forecastValue)
        # tu_denominator += float(pow(actualValue - sequence[-13:-1], 2))
        # tu_molecule += float(pow(actualValue - forecastValue, 2))

        count += 1

        sequence.insert(0, gpno)
        sequence.append(forecastValue)
        result.append(sequence)
        print(count)


    headTrain = ['Train-' + str(i) for i in range(1, len(dataSequence) + 1)]
    headAct = ['Actual']
    headForecast = ['Forecast']
    head = ['统计信息', '数据行数:', count, 'Mean difference:', (allDiffSum / count), 'TU:',
            tu_molecule / (tu_denominator + 0.0000001)]
    firstrow = ['group_no'] + headTrain + headAct + headForecast
    result.insert(0, firstrow)
    result.insert(0, head)
    df_write = pd.DataFrame(result)
    df_write.to_csv(result_file, header=False, index=False, encoding='utf_8_sig')

    print('count: %s' % count)
    print('allDiffSum: %s' % allDiffSum)
    print('Mean difference: %s' % (allDiffSum / count))
    # print('TU: %s' % (tu_molecule / (tu_denominator + 0.0000001)))


if __name__ == '__main__':
    source_floder = r'C:\Users\Administrator\Desktop\LSTM-9分表数据\daset1'
    result_floder = r'C:\Users\Administrator\Desktop\LSTM-9分表数据'
    for root, dirs, files in os.walk(source_floder, topdown=False):
        for filename in files:
            stem, suffix = os.path.splitext(filename)

            source_file = os.path.join(root, filename)
            result_file = os.path.join(result_floder, stem + '-Arima-oneStepResult.csv')
            statistics(source_file, result_file)

预测：[0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29]
1
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
2
预测：[0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06]
3
预测：[0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05]
4


D:\anaconda_3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
5
预测：[0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14]
6
预测：[0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05]
7
预测：[0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.07]
8
预测：[0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05]
9
预测：[0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17 0.17]
10
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
11
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
12
预测：[0.11 0.1  0.09 0.09 0.09 0.09 0.1  0.1  0.1  0.1  0.1  0.1 ]
13
预测：[0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
14
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
15
预测：[0.21 0.19 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15]
16
预测：[0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03]
17
预测：[0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04]
18
预测：[0.15 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23]
19
预测：[0.99 1.01 1.03 1.06 1.08 1.1  1.12 1.14 1.16 1.19 1.21 1.23]
20
预测：[0.14 0.15

D:\anaconda_3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
54
预测：[0.37 0.25 0.18 0.17 0.19 0.27 0.27 0.25 0.24 0.25 0.27 0.28]
55
预测：[0.55 0.55 0.55 0.55 0.55 0.55 0.55 0.55 0.55 0.55 0.55 0.55]
56
预测：[0.67 0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9 ]
57
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
58
预测：[0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04]
59
预测：[0.35 0.36 0.37 0.37 0.38 0.39 0.4  0.41 0.41 0.42 0.43 0.44]
60
预测：[0.08 0.11 0.08 0.08 0.08 0.08 0.09 0.09 0.09 0.09 0.09 0.09]
61
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
62
预测：[0.06 0.43 0.22 0.34 0.27 0.31 0.29 0.3  0.29 0.3  0.3  0.3 ]
63
预测：[0.06 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.06 0.06]
64
预测：[0.24 0.91 0.41 0.35 0.44 0.65 0.67 0.44 0.63 0.6  0.72 0.6 ]
65
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
66
预测：[0.54 0.54 0.54 0.54 0.54 0.54 0.54 0.54 0.54 0.54 0.54 0.54]
67
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
68
预测：[0.26 0.26 0.26 0.26 0.26 0.26 0.26 0.26 0.26 0.26 0.26 0.26]
69
预测：[0.16 0.16 0.16 0.16 0.16 0.1

...

预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
542
预测：[0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.07]
543
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
544
预测：[0.09 0.12 0.13 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14 0.14]
545


D:\anaconda_3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
546
预测：[0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27]
547
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
548
预测：[0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
549
预测：[0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]
550
预测：[0.06 0.08 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06]
551
预测：[0.03 0.03 0.03 0.03 0.03 0.04 0.04 0.04 0.04 0.04 0.04 0.04]
552
预测：[0.24 0.29 0.65 0.35 0.39 0.52 0.4  0.42 0.47 0.42 0.43 0.45]
553
预测：[0.08 0.07 0.05 0.5  0.16 0.15 0.13 0.33 0.21 0.2  0.19 0.27]
554
预测：[0.07 0.06 0.05 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.03 0.03]
555
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
556
预测：[0.05 0.05 0.84 0.03 0.03 0.09 0.09 0.71 0.06 0.06 0.12 0.12]
557
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
558
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
559
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
560
预测：[0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.07]
561
预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
562
预测：[0.0

D:\anaconda_3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


预测：[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
571
预测：[0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06]
572
预测：[0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06 0.06]
573
